In [1]:
import httpx
%load_ext autoreload
%autoreload 2

In [13]:

from farmbase_client.api.farmers import farmers_get_farmer
from farmbase_client.api.gaez import gaez_aez_classification
from farmbase_client import AuthenticatedClient

token = "fsdfds"

latitude=0.14743243681250168
longitude=35.08145448691857

with AuthenticatedClient(base_url="http://127.0.0.1:8000/api/v1", token=token) as client:
    response = await farmers_get_farmer.asyncio(client=client, organization="default", farmer_id=1)
    print(response)

with AuthenticatedClient(base_url="http://127.0.0.1:8000/api/v1", token=token) as client:
    response = await gaez_aez_classification.asyncio(client=client, latitude=latitude, longitude=longitude)
    print(response)


name='John' phone_number='+31657775782' id=1 created_at=datetime.datetime(2025, 5, 6, 10, 12, 1, tzinfo=TzInfo(UTC)) updated_at=datetime.datetime(2025, 5, 6, 10, 12, 1, tzinfo=TzInfo(UTC))
Tropics, lowland; humid


In [5]:
from uuid import UUID
from farmbase_client.models import FieldCreate
from farmbase_client.api.fields import fields_create_field
from geojson_pydantic import LineString

geojson_feature = {
    "type": "LineString",
    "coordinates": [
        [102.0, 0.0], [103.0, 1.0], [104.0, 0.0], [105.0, 1.0]
    ]
}
linestring = LineString(**geojson_feature)
print(linestring)

field = FieldCreate(name="big field",
                    boundary=linestring)
print(field)
with AuthenticatedClient(base_url="http://127.0.0.1:8000", token=token) as client:
    response = fields_create_field.sync(client=client, body=field,
                                        farm_id=UUID('b3d366ef-6398-4f1e-8cf4-566bf216fcba'))

response

bbox=None type='LineString' coordinates=[Position2D(longitude=102.0, latitude=0.0), Position2D(longitude=103.0, latitude=1.0), Position2D(longitude=104.0, latitude=0.0), Position2D(longitude=105.0, latitude=1.0)]
name='big field' area=10.5 soil_type='clay' boundary=LineString(bbox=None, type='LineString', coordinates=[Position2D(longitude=102.0, latitude=0.0), Position2D(longitude=103.0, latitude=1.0), Position2D(longitude=104.0, latitude=0.0), Position2D(longitude=105.0, latitude=1.0)])


FieldPublic(name='big field', area=None, soil_type=None, id=UUID('13587135-d7a1-4d49-a56d-c357af475463'), farm_id=UUID('b3d366ef-6398-4f1e-8cf4-566bf216fcba'), boundary=Polygon(bbox=None, type='Polygon', coordinates=[[Position2D(longitude=102.0, latitude=0.0), Position2D(longitude=103.0, latitude=1.0), Position2D(longitude=104.0, latitude=0.0), Position2D(longitude=105.0, latitude=1.0), Position2D(longitude=102.0, latitude=0.0)]]))

In [1]:
import pandas as pd

df = pd.read_csv("apps/farmbase/data/kalro/suitability.csv")

In [3]:
df.to_pickle("apps/farmbase/data/kalro/suitability.pkl")

In [32]:
df[df.crop == 'Kale']

,crop,variety,dhis2_id,suitability


In [16]:
df[df.crop == 'Barley'].pivot(index='dhis2_id', columns='variety', values='suitability')

variety,ALICIANA,Ahadi,BAHATI,Bima,CERISE LAUREL,COCKTAIL,GRACE,KARNE,NFC TIPPLE,NGAO,PEBLICAN,Planet,QUENCH,Sabini,Tumaini
dhis2_id,,,,,,,,,,,,,,,
A34NiI4rgnf,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
A6Sj8RumZ0m,1.0,NaN,NaN,1.0,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,NaN
ABuzigW8Lzw,1.0,NaN,NaN,1.0,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0
ABy8CNqf2e7,1.0,NaN,NaN,1.0,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,NaN
AFIlzKNxXST,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ylWg6VvADJE,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0
zCCIu1Vi3Wh,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
zI6vnsXresW,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
import httpx
r = httpx.get("http://127.0.0.1:8000/api/v1/crop-varieties/suitability?crop_type=kale"
              "&latitude=-1.0849843025208&longitude=35.878883361816")
r.json()

{'detail': "No crops found matching 'kale'. Did you mean one of these: Kales, Barley?"}